# Why would you use batch gradient descent vs stochastic gradient descent?
# When do you know to use a particular optimization algorithm. 

These two questions are related, I'll answer both together.

The long discussion is here:
https://stats.stackexchange.com/questions/49528/batch-gradient-descent-versus-stochastic-gradient-descent

This is a classic runtime vs optimality situation. Batch is more optimal at the cost of long run time for large datasets, and stochastic is quick and dirty.

Thus, you will want to use batch if optimality is your primary concern, and stochastic if runtime is your concern. If you have a huge dataset and you are just exploring it, then maybe go with stochastic. If you are able to reduce or otherwise pre-process your dataset, then perhaps batch may become viable.

A more technical discussion involves discussing how batch operates vs stochastic, specifically, with respect to the computation of the cost function. Batch is so costly because it processes every data point during each iteration of the cost function computation. The advantage is that batch guarantees it will converge to the global minimum for convex error surfaces and to a local minimum for non-convex surfaces. This optimality comes at the cost of high runtime.

With stochastic, the gradient of the loss function is computed with a single random data point rather than the entire data set, as in batch. This is how stochastic immediately begins addressing the minimization. The disadvantages are that stochastic results in a larger variance of the loss function than batch, and also that it cannot guarantee global or local minima.

# What is dependency injection?


Dependency injection is an OOP concept. It helps answer some of the following questions:
How can an application be independent of how its objects are created?
How can a class be independent of how the objects it requires are created?
How can the way objects are created be specified in separate configuration files?
How can an application support different configurations?

Dependency injection is the idea that you can decouple objects within a class such that the class's code doesn't have to be changed if any of the objects are changed or switched out. In other words, the class doesn't produce values within itself using "static" or "new" methods; all these values must be "injected" from outside the class by objects that are passed in.

This 

# Compare the number of iterations it takes for each algorithm to obtain an estimate accurate to 1e-3 
(you may wish to set a cap for maximum number of iterations). Which method converges to the optimal point in fewer iterations? Briefly explain why this result should be expected. 

## First let's look at batch descent, then stochastic, then answer the question afterwards. Precision will go to 0.001 in both cases.

In [4]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats 
from sklearn.datasets.samples_generator import make_regression 

In [8]:
# code from:
# https://am207.github.io/2017/wiki/gradientdescent.html
def gradient_descent(x, y, theta_init, step=0.001, maxsteps=0, precision=0.001, ):
    costs = []
    m = y.size # number of data points
    theta = theta_init
    history = [] # to store all thetas
    preds = []
    counter = 0
    oldcost = 0
    pred = np.dot(x, theta)
    error = pred - y 
    currentcost = np.sum(error ** 2) / (2 * m)
    preds.append(pred)
    costs.append(currentcost)
    history.append(theta)
    counter+=1
    while abs(currentcost - oldcost) > precision:
        oldcost=currentcost
        gradient = x.T.dot(error)/m 
        theta = theta - step * gradient  # update
        history.append(theta)
        
        pred = np.dot(x, theta)
        error = pred - y 
        currentcost = np.sum(error ** 2) / (2 * m)
        costs.append(currentcost)
        
        if counter % 25 == 0: preds.append(pred)
        counter+=1
        if maxsteps:
            if counter == maxsteps:
                break
        
    return history, costs, preds, counter

x, y = make_regression(n_samples = 100, 
                       n_features=1, 
                       n_informative=1, 
                       noise=20,
                       random_state=2017)
x = x.flatten()
slope, intercept, _,_,_ = stats.linregress(x,y)
best_fit = np.vectorize(lambda x: x * slope + intercept)

xaug = np.c_[np.ones(x.shape[0]), x]
theta_i = [-15, 40] + np.random.rand(2)
history, cost, preds, iters = gradient_descent(xaug, y, theta_i, step=0.001)
theta = history[-1]
print("Gradient Descent: {:.2f}, {:.2f} {:d}".format(theta[0], theta[1], iters))
print("Least Squares: {:.2f}, {:.2f}".format(intercept, slope))

Gradient Descent: -3.93, 81.67 4454
Least Squares: -3.71, 82.90


## Now, let's look at stochastic descent:

In [10]:
def sgd(x, y, theta_init, step=0.001, maxsteps=0, precision=0.001, ):
    costs = []
    m = y.size # number of data points
    oldtheta = 0
    theta = theta_init
    history = [] # to store all thetas
    preds = []
    grads=[]
    counter = 0
    oldcost = 0
    epoch = 0
    i = 0 #index
    pred = np.dot(x[i,:], theta)
    error = pred - y[i]
    gradient = x[i,:].T*error
    grads.append(gradient)
    print(gradient,x[i],y[i],pred, error, np.sum(error ** 2) / 2)
    currentcost = np.sum(error ** 2) / 2
    counter+=1
    preds.append(pred)
    costsum = currentcost
    costs.append(costsum/counter)
    history.append(theta)
    print("start", counter, costs, oldcost)
    while 1:
        #while abs(costs[counter-1] - oldcost) > precision:
        #while np.linalg.norm(theta - oldtheta) > precision:
        #print("hi", precision)
        #oldcost=currentcost
        gradient = x[i,:].T*error
        grads.append(gradient)
        oldtheta = theta
        theta = theta - step * gradient  # update
        history.append(theta)
        i += 1
        if i == m:#reached one past the end.
            #break
            epoch +=1
            neworder = np.random.permutation(m)
            x = x[neworder]
            y = y[neworder]
            i = 0
        pred = np.dot(x[i,:], theta)
        error = pred - y[i]
        currentcost = np.sum(error ** 2) / 2
        
        #print("e/cc",error, currentcost)
        if counter % 25 == 0: preds.append(pred)
        counter+=1
        costsum += currentcost
        oldcost = costs[counter-2]
        costs.append(costsum/counter)
        #print(counter, costs, oldcost)
        if maxsteps:
            #print("in maxsteps")
            if counter == maxsteps:
                break
        
    return history, costs, preds, grads, counter, epoch

history2, cost2, preds2, grads2, iters2, epoch2 = sgd(xaug, y, theta_i, maxsteps=5000, step=0.001)


(array([-25.11218394,  -0.80995399]), array([ 1.        ,  0.03225343]), 11.534851890155354, -13.577332051876045, -25.1121839420314, 315.31089116920987)
('start', 1, [315.31089116920987], 0)


## ANSWER:
For a precision of 0.001, it took batch 4454 iterations and stochastic only ~316. This is expected, because stochastic is quick and directly, getting to the answer much more quickly.

# Compare the performance of stochastic gradient descent for the following learning rates: 1, 0.1, 0.001, 0.0001. 
Based on your observations, briefly describe the effect of the choice of learning rate on the performance of the algorithm.


In [11]:
history2, cost2, preds2, grads2, iters2, epoch2 = sgd(xaug, y, theta_i, maxsteps=5000, step=1)
history2, cost2, preds2, grads2, iters2, epoch2 = sgd(xaug, y, theta_i, maxsteps=5000, step=0.1)
history2, cost2, preds2, grads2, iters2, epoch2 = sgd(xaug, y, theta_i, maxsteps=5000, step=0.001)
history2, cost2, preds2, grads2, iters2, epoch2 = sgd(xaug, y, theta_i, maxsteps=5000, step=0.0001)

(array([-25.11218394,  -0.80995399]), array([ 1.        ,  0.03225343]), 11.534851890155354, -13.577332051876045, -25.1121839420314, 315.31089116920987)
('start', 1, [315.31089116920987], 0)
(array([-25.11218394,  -0.80995399]), array([ 1.        ,  0.03225343]), 11.534851890155354, -13.577332051876045, -25.1121839420314, 315.31089116920987)
('start', 1, [315.31089116920987], 0)
(array([-25.11218394,  -0.80995399]), array([ 1.        ,  0.03225343]), 11.534851890155354, -13.577332051876045, -25.1121839420314, 315.31089116920987)
('start', 1, [315.31089116920987], 0)
(array([-25.11218394,  -0.80995399]), array([ 1.        ,  0.03225343]), 11.534851890155354, -13.577332051876045, -25.1121839420314, 315.31089116920987)
('start', 1, [315.31089116920987], 0)


## ANSWER:
It looks like changing the learning rate from 1 to 0.0001 doesn't seem to change any of the performance metrics.